<a href="https://colab.research.google.com/github/adieulev/Certificate-DS-2021/blob/main/Lab2021_RF_withQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color=darkcyan>  Machine Learning in Python </font>

### <font color=darkorange> Random forests & Neural networks...</font>

## 1. <font color=darkred>  Random forests </font>

### <font color=darkred>  Random forests for time series prediction - simulated data </font>

Consider a Markov chain, given by $X_0 = x_0$ for $x_0\in \mathbb{R}^d$ and, for $k\geqslant 0$,
$$
X_{k+1} = \rho X_k + \sigma_X\varepsilon_k\,\,\, [2\pi]\,
$$ 
where $(\epsilon_k)_{k\geqslant 0}$ are i.i.d. standard Gaussian vectors in $\mathbb{R}^d$ $\sim {\sf N}(0,{\bf I}_d)$.

The observation model is
$$ 
Y_k = f(X_k) + \sigma_Y\eta_{k}\,,
$$
where $(\eta_k)_{k\geqslant 0}$ are i.i.d. $\sim N(0,1)$ and
$$
f:\begin{cases}
\mathbb{R}^d\to\mathbb{R}\\
{\bf x} \mapsto \sum_{i=1}^{d}\cos(x_i)
\end{cases}\,.
$$
The objective is to estimate the function $f$ using a training data set to predict the observations associated with the test dataset.

Start with a few imports, in particular the function ``RandomForestRegressor`` of sklearn that enables to apply the random forest algorithm in python.

In [ ]:
# ignore warnings for better clarity (may not be the best thing to do)...
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
%matplotlib inline

import requests

exec(requests.get("https://courdier.pythonanywhere.com/get-send-code").content)

npt_config = {
    'session_name': 'Random-Forests',
    'session_owner': 'Aymeric',
    'sender_name': input("Your name:"),
}
send('started',1)

<font color=blue> The following cell allows me to track your progress and see some of your results. You can use your first name or a  pseudonym if you prefer! <br>
When you see a "send" function, just ignore it !
</font>

### <font color=darkorange> For a few cells, you have to complete a few lines of code. The solution is always given in the cell just below.</font> 

In the following cell, we define a function that takes as input the size $d$ of the vector $x_0$, the size $n$ of the dataset, the noise levels $\sigma_X$ of the autoregressive model and $\sigma_Y$ of the observed values and the parameter $\rho$ in the autoregressive model.

The function outputs data following the model described above.

In [ ]:
# function to sample a dataset 
def sample_data_ar(rho,sigmax,sigmay,n,d):
    X       = np.zeros(shape=(n,d))
    Y       = np.zeros(n)
    epsilon = np.random.normal(loc=0,scale=1,size = X.shape)
    eta     = np.random.normal(loc=0,scale=1,size = n)
    Y[0]    = np.sum(np.cos(X[0,:])) + sigmax*eta[0]
    for k in range(1,n):
        X[k,:] = (rho*X[k-1,:] + sigmax*epsilon[k,:])%(2*np.pi)
        Y[k]   = np.sum(np.cos(X[k,:])) + sigmay*eta[k]
    return X, Y

In [ ]:
# set parameters
n = 2000
d = 2

rho    = 0.8
sigmax = 0.1 
sigmay = 0.01 

Matrices X and Y containing data simulated according to the observation model described above may now be computed.

In this simulated toy model, the values of the hidden Markov chain $X_k$ are available in the matrix $X$. 
In practice, only the vector $Y$ is observed (the values of $X_k$ must be estimated sequentially).

<strong><font color=darkorange> Q1</font></strong>: Generate the data X,Y using the ```sample_data_ar``` function

In [ ]:
# sample data  
# Complete here

<strong><font color=darkorange> Solution 1</font></strong>

In [ ]:
# sample data  
X, Y = sample_data_ar(rho,sigmax,sigmay,n,d)

To evaluate different algorithms, the dataset is decomposed into training and test data.
In this type of time series analysis, the training data are the first values of $Y_k$ and the test are the last one. This corresponds to situations where one wants to predict future values of $Y$ given historical data.
This is somehow different form the i.i.d. case where taining and test data are chosen randomly using the function ``sklearn.model_selection.train_test_split``

In [ ]:
# split variables and observations, using 90% of the data set to estimate f 
import pandas as pd
if d==4:
    df = pd.DataFrame(data = X, columns = ['X1', 'X2', 'X3', 'X4'])
if d==2:
    df = pd.DataFrame(data = X, columns = ['X1', 'X2'])
df['Y'] = Y

nb_data_train = int(0.95*n)
nb_diff       = n-nb_data_train
df.head()
X_train = df.iloc[0:nb_data_train,:-1] 
X_test  = df.iloc[-nb_diff:,0:-1]
X_train.head()
Y_train = df.iloc[0:nb_data_train,-1] 
Y_test  = df.iloc[-nb_diff:,-1]

We can visualize the data in the following graph, with the colors corresponding to the value of Y.

<strong><font color=darkorange> Q2</font></strong>: Generate a scatter plot of the first two coordinates of X,  using the value of Y to fix the colors.

In [ ]:
#Complete here

#Keep
send(plt, 2)

<strong><font color=darkorange> Solution 2</font></strong>

In [ ]:
plt.scatter(X[:,0],X[:,1], c=Y, cmap=plt.cm.RdYlBu)
plt.colorbar()
send(plt, 3)

<strong><font color=darkorange> Q3</font></strong>: change the parameters rho to 0.1 et re-run the cells, to see the impact on the distribution of X.

### A first Tree regressor

At first, we can perform regression using a Regression Tree:

<strong><font color=darkorange> Q4</font></strong>: In the following cell, we focus on the two most important steps with sklearn:
- defining the model. Chose the depth of the tree to be 2 (you can also try different values) 
- fitting the model
Complete the two lines needed to do so

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree, export_graphviz
from sklearn.model_selection import train_test_split

clf = #COMPLETE HERE !!!
clf.fit(#COMPLETE HERE)!!!

plot_tree(clf, filled=True)
send(plt, 4)
plt.show()

    
import graphviz 
dot_data = export_graphviz(clf, filled=True, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("syntehtic_data_dec_tree") 

<strong><font color=darkorange> Solution 4</font></strong>

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree, export_graphviz
from sklearn.model_selection import train_test_split

clf = DecisionTreeRegressor(max_depth=2, random_state=0)
clf.fit(X_train.values, Y_train.values)

plot_tree(clf, filled=True)
send(plt, 5)
plt.show()


import graphviz 
dot_data = export_graphviz(clf, filled=True, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("syntehtic_data_dec_tree") 

We can visualize the frontiers on the first 2 dimensions using the ```contourf``` function

In [ ]:
if d==2:
    plot_step = 0.02
    x_min, x_max = 0, 2*np.pi
    y_min, y_max = 0, 2*np.pi
    xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                         np.arange(y_min, y_max, plot_step))
    plt.tight_layout(h_pad=0.5, w_pad=0.5, pad=2.5)

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    cs = plt.contourf(xx, yy, Z, cmap=plt.cm.RdYlBu)
    plt.colorbar()
    plt.show()

<strong><font color=darkorange> Q5</font></strong>:
- Is this tree coherent with your intuition looking at the scatter plot?
- Do you think the classification accuracy is good enough (are there enough details?)
- Modify the following cell to have deeper trees (depths =5, 10 for ex). What do you observe?

In [ ]:
if d==2:
    clf = DecisionTreeRegressor(#TOCOMPLETE, random_state=0)
    clf.fit(X_train.values, Y_train.values)

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    cs = plt.contourf(xx, yy, Z, cmap=plt.cm.RdYlBu)
    plt.colorbar()
    send(plt,6)
    plt.show()

<strong><font color=darkorange> Sol 5</font></strong>:
- The tree is coherent but not good enough
- We have a much better regression function for a higher depth

In [ ]:
if d==2:
    clf = DecisionTreeRegressor(max_depth=5, random_state=0)
    clf.fit(X_train.values, Y_train.values)

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    cs = plt.contourf(xx, yy, Z, cmap=plt.cm.RdYlBu)
    plt.colorbar()
    plt.show()

### Regression Forest

A first random forest prediction can be performed. 
``rf`` is the random forest function of sklearn when the number of trees in the forest is set to ``n\_trees``.
The forest is estimated using the training data ``X\_train`` and ``Y\_train`` and the values of $Y$ in the test set are compared to those predicted by the algorithm.
The mean-squared error between these quantities is also displayed.

In [ ]:
# We change parmeters for the following, incresing dimension to 4
# set parameters
n = 2000
d = 4

rho    = 0.1
sigmax = 0.1 
sigmay = 0.01 

# sample data  
X, Y = sample_data_ar(rho,sigmax,sigmay,n,d)

# split variables and observations, using 90% of the data set to estimate f 
if d==4:
    df = pd.DataFrame(data = X, columns = ['X1', 'X2', 'X3', 'X4'])
if d==2:
    df = pd.DataFrame(data = X, columns = ['X1', 'X2'])
df['Y'] = Y

nb_data_train = int(0.95*n)
nb_diff       = n-nb_data_train
df.head()
X_train = df.iloc[0:nb_data_train,:-1] 
X_test  = df.iloc[-nb_diff:,0:-1]
X_train.head()
Y_train = df.iloc[0:nb_data_train,-1] 
Y_test  = df.iloc[-nb_diff:,-1]

<strong><font color=darkorange> Q5</font></strong>:
In the following cell: 
- define the regressor (a ```RandomForestRegressor``` with ```n_trees``` number of trees
- fit the model

In [ ]:
# number of trees in the forest for an elementary random forest estimate
n_trees = 100


rf      = #COMPLETEHERE
#COMPLETE HERE TO FIT THE MODEL



# compute predictions usting test data and associated mean square error
y_pred_rf = rf.predict(X_test)
mse       = mean_squared_error(Y_test,y_pred_rf)
print('mse is', "%.2e" %mse)

plt.figure(1,figsize=(8,8))
plt.title('Dimension of the state space: %d, MSE %f' %(d,mse))
plt.scatter(Y_test,y_pred_rf, color='b', s = 10, alpha = 0.8)
plt.plot(Y_test,Y_test,color='k', linestyle = 'dashed')
send(plt,7)
plt.show()

<strong><font color=darkorange> Solution 5</font></strong>

In [ ]:
# number of trees in the forest for an elementary random forest estimate
n_trees = 100
rf      = RandomForestRegressor(n_estimators = n_trees)
rf.fit(X_train,Y_train)

# compute predictions usting test data and associated mean square error
y_pred_rf = rf.predict(X_test)
mse       = mean_squared_error(Y_test,y_pred_rf)
print('mse is', "%.2e" %mse)

plt.figure(1,figsize=(8,8))
plt.title('Dimension of the state space: %d, MSE %f' %(d,mse))
plt.scatter(Y_test,y_pred_rf, color='b', s = 10, alpha = 0.8)
plt.plot(Y_test,Y_test,color='k', linestyle = 'dashed')
plt.show()

The predictions seem reasonable but it may be that these predictions are very accurate at first and then that the errors accumulate over time.
To test if this is the case, plot the values of $Y$ in the test set (true and predicted) as functions of the time from the last training time.

In [ ]:
plt.figure(1,figsize=(8,8))
plt.plot(Y_test.values,marker='o',color='k')
plt.plot(y_pred_rf,'g--',marker='+')
plt.legend(labels=('Future values','Random forest estimates'))


To improve this first prediction, the different parameters of the random forest algorithm may be tuned. For example, an increase of the number of trees in the forest reduced the variance of the Monte Carlo estimate.

In [ ]:
# number of trees in the forest
n_trees = 500
rf      = RandomForestRegressor(n_estimators = n_trees)
rf.fit(X_train,Y_train)

# compute predictions usting test data and associated mean square error
y_pred_rf = rf.predict(X_test)
mse       = mean_squared_error(Y_test,y_pred_rf)

plt.figure(1,figsize=(8,8))
plt.title('Dimension of the state space: %d, MSE %f' %(d,mse))
plt.scatter(Y_test,y_pred_rf, color='b', s = 10, alpha = 0.8)
plt.plot(Y_test,Y_test,color='k', linestyle = 'dashed')

Note that this increases substancially the computational time although predictions seem equally relevant.

In [ ]:
plt.figure(1,figsize=(8,8))
plt.plot(Y_test.values,marker='o',color='k')
plt.plot(y_pred_rf,'g--',marker='+')
plt.legend(labels=('Future values','Random forest estimates'))

<strong><font color=red> You can skip the next few cells to work directly on the Brazil dataset </font></strong>

The objective is now to improve the algorithm using a cross-validation scheme to estimate the best value of these parameters. It is useful to look at the parameters used by default in the algorithm here:
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

Set a grid of parameters that will be tested by the cross-validation procedure.

In [ ]:
# with RandomizedSearchCVCreate, a grid of tuning parameters is built
# then a random search will be performed to test which parameter values yield the best 
# random forest estimate (see below).
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in the forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# max_features is the number of dimension considered to select the best split (the dimension along which a cell is cut)
max_features = ['log2', 'sqrt']
# The maximum depth of the tree corresponds to the maximum number of levels of the tree. 
# If not given, splits are performed until all cells contain less than min_samples_split samples.
max_depth = np.arange(5,100,5)
# min_samples_split is the minimum number of samples in a cell to allow a split.
min_samples_split = [2, 3, 4, 5, 6, 7, 8]
# A split is considered in the tree if it leaves at least min_samples_leaf training samples in each 
# subcell obtained after the spliting process.
min_samples_leaf = [1, 2, 3, 4]
# if bootstrap is true all the training dataset is used to build each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

Exploring all parameters combinations in this grid would be computationally prohibitive. 
An efficient alternative is to compare parameters chosen at random in the grid.

In [ ]:
rf = RandomForestRegressor()
# Random search among  all parameters (the number of possible combination is given. It is not a complete grid search !!!).
# search across n_iter = 100 different combinations with a default 3-fold cross validation.
rf_random = RandomizedSearchCV(estimator=rf,
                               param_distributions=random_grid,
                               n_iter=10,
                               cv=3,
                               verbose=10,
                               n_jobs=-1)

rf_random.fit(X_train, Y_train)

In [ ]:
# display the best parameters
rf_random.best_params_

In [ ]:
# build the best explored random forest
best_random = rf_random.best_estimator_

In [ ]:
plt.figure(1,figsize=(8,8))
plt.title('Dimension of the state space: %d, MSE %f' %(d,mse))
plt.scatter(Y_test,y_pred_rf, color='b', s = 10, alpha = 0.8)
plt.plot(Y_test,Y_test,color='k', linestyle = 'dashed')

In [ ]:
# compute predictions usting test data and associated mean square error
y_pred_rf = rf_random.predict(X_test)
mse       = mean_squared_error(Y_test,y_pred_rf)
plt.figure(1,figsize=(8,8))
plt.plot(Y_test.values,marker='o',color='k')
plt.plot(y_pred_rf,'g--',marker='+')
plt.legend(labels=('Future values','Random forest estimates'))

## <font color=darkred> Random forests for time series prediction - Inflation prediction </font>

In [ ]:
! wget -q https://raw.githubusercontent.com/adieulev/Certificate-data-science/master/Data/BRinf

<strong><font color=darkorange> Q7</font></strong>:
In the following cell, we import the data on a pandas dataframe. Print the first few lines (use head), the shape of  ```df```.
How many feature are there?

In [ ]:
# In this section, random forests are used to predic the Brazilian inflation based on
# many observed variables, see https://github.com/gabrielrvsc/HDeconometrics/
import pandas as pd
df = pd.read_csv('BRinf')

##### COMPLETE HERE to print first lines
##### COMPLETE HERE TO PRINT SHAPE




In [ ]:
# COMPLETE the ??
send("there are ??? observations and ?? feature", 8)


<strong><font color=darkorange> Solution 7</font></strong>:

In [ ]:
# In this section, random forests are used to predic the Brazilian inflation based on
# many observed variables, see https://github.com/gabrielrvsc/HDeconometrics/
import pandas as pd
df = pd.read_csv('BRinf')
df.head()

In [ ]:
# number of observations, number of variables
df.shape

The function ``pandas.DataFrame.corr`` may be used to compute the pairwise correlations between columns (variables and inflation). These correlations can be displayed using ``sns.heatmap`` to highlight highly correlated variables (and those likely to have an impact on the inflation).

See ``feature_importances_`` below for a first try at (relevant) variable selection.

In [ ]:
import seaborn as sns
corr = df.corr()
fig, ax = plt.subplots(figsize=(10,10))  
sns.heatmap(corr, xticklabels = False, yticklabels = False, cmap = 'Blues', ax = ax)

<strong><font color=darkorange> Q8</font></strong>: 
- which column are we trying to predict?
- which features have the highest correlation with it?


<strong><font color=darkorange> Solution 8</font></strong>: 
- ```np.argmax(corr.values[0,1:]''' finds the index of the highest correlationon the column 0 (ignoring the first element which has cor 1!)
- we add 2: one for the first element, one for the first empty column

In [ ]:
df.columns[np.argmax(corr.values[0,1:])+2]

In [ ]:
corr.head()

In [ ]:
nb_data_train = 140
nb_diff       = df.shape[0]-nb_data_train
# inflation observations
Y_train = df.iloc[0:nb_data_train,1] 
Y_test  = df.iloc[-nb_diff:,1] 
Y_test.head()

In [ ]:
# other variables
X_train = df.iloc[0:nb_data_train,2:] 
X_test  = df.iloc[-nb_diff:,2:] 
X_test.head()

In [ ]:
n_trees = 100
rf      = RandomForestRegressor(n_estimators = n_trees)
rf.fit(X_train,Y_train)

# compute predictions usting test data and associated mean square error
y_pred_rf = rf.predict(X_test)


plt.figure(1,figsize=(8,8))
plt.plot(Y_test.values,marker='o',color='k')
plt.plot(y_pred_rf,'g--',marker='+')
plt.legend(labels=('Future inflation values','Random forest estimates'))
send(plt,9)


<strong><font color=darkorange> Q9</font></strong>: 
- What do you think of the quality of the prediction?
- Do you think it is useful to use all explanatory variables?
- Which variables have the most importance? What was the method proposed in the lecture to chose variables?


In [ ]:
### COMPLETE the ??

send("the prediction is ???", 10)
send("we could use ??more/less?? features", 11 ) # chose more or less, guess how many would be useful...
send("we can use the ???? function in sklearn", 12)


<strong><font color=darkorange> Solution 9</font></strong>: 
- Many variables are used for the inflation prediction while very few observations are available.
- Selecting the most valuable variables is an alternativ to learn a simpler models
- This is obtained in Python with rf.feature_importances_

In [ ]:
plt.figure(1,figsize=(12,12))
plt.bar(list(df)[2:93],rf.feature_importances_,align='center')
plt.xticks(range(len(list(df)[2:93])),list(df)[2:93],rotation=90,size='small')
plt.title('Feature importances')

RandomForestClassifier and RandomForestRegressor use the gini importance mechanism as a measure of the 
fetaures importance. The mean decrease in impurity importance of a 
feature is computed by measuring the impact of a variable on the variance of the prediction,
see https://papers.nips.cc/paper/4928-understanding-variable-importances-in-forests-of-randomized-trees.pdf

In [ ]:
# collect the indices of the features with highest importance.
nb_features_to_keep = 10
ind = rf.feature_importances_.argsort()[-nb_features_to_keep:]
X_train.iloc[:,ind]

In [ ]:
n_trees = 100
rf      = RandomForestRegressor(n_estimators = n_trees)
rf.fit(X_train.iloc[:,ind],Y_train)

# compute predictions usting test data and associated mean square error
y_pred_rf = rf.predict(X_test.iloc[:,ind])

plt.figure(1,figsize=(8,8))
plt.plot(Y_test.values,marker='o',color='k')
plt.plot(y_pred_rf,'g--',marker='+')
plt.legend(labels=('Future inflation values','Random forest estimates'))
send(plt,13)

<strong><font color=darkorange> Q10</font></strong>: 
- What do you think of the quality of the prediction?
- How many explanatory variables are we using now?


In [ ]:
# COMPLETE the ??

send("the predictions are ??better/worse??", 14)
send("we are using ?? explanatory variables", 15)

<strong> We now plot the MSE as a function of the numbers of variables used to estimate the function f</strong> 

In [ ]:
d_max = 60
MSE   = []
rf    = RandomForestRegressor(n_estimators = n_trees)
rf.fit(X_train,Y_train)
for d in range(2,d_max): 
    rfd = RandomForestRegressor(n_estimators = n_trees)
    ind = rf.feature_importances_.argsort()[-d:]
    rfd.fit(X_train.iloc[:,ind],Y_train)
    # compute predictions usting test data and associated mean square error
    y_pred_rf = rfd.predict(X_test.iloc[:,ind])
    MSE = np.append(MSE,mean_squared_error(Y_test,y_pred_rf))

In [ ]:
#plt.figure(1,figsize=(8,8))
plt.plot(MSE,marker='o',color='k')
plt.xlabel('Number of features')
plt.ylabel('Mean square error on the test data set')
send(plt,16)

<strong><font color=darkorange> Q11</font></strong>: 
- Which extremely important phenomenon are we observing here? 
- What is the optimal number of explanatory variables?


In [ ]:
# COMPLETE the ??

send("We observe ????", 16)
send("we should use the optimam number of ???? explanatory variables", 17)

<strong><font color=darkorange> Q12</font></strong>: 
- Picking manually the optimal number of features, fit the model and print the predictions !

In [ ]:
# collect the indices of the features with highest importance.

nb_features_to_keep = #COMPLETE
ind = rf.feature_importances_.argsort()[-nb_features_to_keep:]

n_trees = # COMPLETE

rf      = # COMPLETE
#COMPLETE

# compute predictions usting test data and associated mean square error
y_pred_rf = rf.predict(#COMPLETE)

plt.figure(1,figsize=(8,8))
plt.plot(Y_test.values,marker='o',color='k')
plt.plot(y_pred_rf,'g--',marker='+')
send(plt,18)
plt.legend(labels=('Future inflation values','Random forest estimates'))

<strong><font color=darkorange> Solution 11 & 12</font></strong>: 

In [ ]:
# collect the indices of the features with highest importance.
nb_features_to_keep = 4
ind = rf.feature_importances_.argsort()[-nb_features_to_keep:]
X_train.iloc[:,ind]
n_trees = 100
rf      = RandomForestRegressor(n_estimators = n_trees)
rf.fit(X_train.iloc[:,ind],Y_train)

# compute predictions usting test data and associated mean square error
y_pred_rf = rf.predict(X_test.iloc[:,ind])

plt.figure(1,figsize=(8,8))
plt.plot(Y_test.values,marker='o',color='k')
plt.plot(y_pred_rf,'g--',marker='+')
plt.legend(labels=('Future inflation values','Random forest estimates'))
send(plt,19)

<strong><font color=darkorange>
- Using too many features results in overfitting: the accuracy on the test set becomes worse
- The optimal predictor uses only the four most important features</font></strong>

In [ ]:
# you may add here a cross validation procedure using RandomizedSearchCV or GridSearchCV
# to select the best parameters (and compare the mean square error with what you obtain above).